In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importamos librerías
import gzip, json, re, random
import pandas as pd

Limpieza y balanceado de datos
Convertir a CSV

In [ ]:
#Ruta al archivo de reseñas
input_path = '/content/drive/MyDrive/Proyecto_Grupo4/Cell_Phones_and_Accessories.json.gz'

# Nombre de columna con texto y puntaje en el JSON
text_col  = "reviewText"   # cambia si tu JSON usa otro nombre (p.ej. 'review_body')
score_col = "overall"      # Puntuacion del 1 al 5

# Tamaño objetivo por clase, con el fin de obtener un buen balance para el analisis
target_per_class = {
    "negative": 10_000, "neutral":  10_000, "positive": 10_000
}

# Archivo de salida
out_csv = "amazon_reviews_limpio_balanceado.csv"

# Palabras clave (en inglés) para detección simple de aspectos
KW_PRICE = [
    "price", "expensive", "cheap", "cost", "costly", "affordable", "value", "worth"
]
KW_QUALITY = [
    "quality", "defect", "defective", "broken", "poor", "bad", "good", "excellent",
    "durable", "sturdy", "flimsy", "faulty"
]
KW_SHIPPING = [
    "shipping", "delivery", "arrived", "late", "delay", "delayed", "package",
    "packaging", "courier", "carrier"
]

random.seed(42) # Semilla para reproducibilidad, fijar secuencia de numeros aleatorios

#Definir overall a Sentimientos.
def map_sentiment(score):
    try:
        s = float(score)
    except:
        return None #Devuelve None si el score no es numérico.
    if s <= 2:
        return "negative"
    if s == 3:
        return "neutral"
    return "positive"
#Limpieza, convierte a str, comprime espacios, recorta.
def clean_text(text):
    t = str(text or "")
    t = re.sub(r"\s+", " ", t).strip()
    return t
#Devuelve True si alguna palabra clave aparece como substring
def has_kw(text, keywords):
    t = text.lower()
    return any(kw in t for kw in keywords)

# Obtener muestras que sean representativas y aleatorios del conjunto total, para obtener un buen analisis
reservoir = {k: [] for k in target_per_class}  # items guardados por clase
seen = {k: 0 for k in target_per_class}        # cuántos vimos por clase en total

def maybe_add_to_reservoir(item, label):
    #Reservoir sampling por clase:
    #- Si aún no llenamos el cupo, agregamos.
    #- Si ya está lleno, reemplazamos al azar con prob k/seen para mantener aleatoriedad.
    seen[label] += 1
    k = target_per_class[label]
    bucket = reservoir[label]
    if len(bucket) < k:
        bucket.append(item)
    else:
        j = random.randint(0, seen[label] - 1)
        if j < k:
            bucket[j] = item
    #Lectura de los archivos
print("Leyendo el archivo JSON Lines…")
with gzip.open(input_path, "rt", encoding="utf-8") as f: #Abrir el archivo, leer como texto
    for ln, line in enumerate(f, start=1): #linea por linea
        line = line.strip() #Suprime espacios o saltos de lineas al comienzo y final
        if not line: #Se salta la linea queda vacía
            continue
        # Cada línea es un objeto JSON
        try:
            obj = json.loads(line) #Convierte la linea a diccionario python
        except json.JSONDecodeError:
            continue  # saltar líneas corruptas

        text = clean_text(obj.get(text_col)) #Obtiene el texto de reseña y lo normaliza
        score = obj.get(score_col, None) #OObtiene la puntuación del 1 al 5
        if not text or score is None: #Si falta puntuación se salta
            continue

        label = map_sentiment(score) #La columna score  convierte en negativo, neutral, positivo.
        if label is None: #Si no tiene score se salta
            continue

        # Flags de aspectos columnas claves con relaciones, si las reseñas menciona precio, calidad o envío
        mention_price    = has_kw(text, KW_PRICE)
        mention_quality  = has_kw(text, KW_QUALITY)
        mention_shipping = has_kw(text, KW_SHIPPING)

        # Guardar (o reemplazar) en reservoir de su clase si es válido para obtener un balanceo por clase
        item = (text, label, float(score), mention_price, mention_quality, mention_shipping)
        maybe_add_to_reservoir(item, label)

        # Progreso cada 1M de líneas
        if ln % 1_000_000 == 0: #Cada millon de líneas leídas imprime lineas totales y reseñas guardadas por clase
            counts = {k: len(v) for k, v in reservoir.items()}
            print(f"Leídas {ln:,} líneas | guardadas -> neg={counts['negative']:,}, "
                  f"neu={counts['neutral']:,}, pos={counts['positive']:,}")

        # Parada temprana cuando se llenan las 3 clases, que cada clase tengas las muestras necesarias
        if all(len(reservoir[lbl]) >= target_per_class[lbl] for lbl in target_per_class):
            print(f"\n Muestras completas después de leer {ln:,}.")
            break
print("Lectura finalizada.")

#Construir DataFrame balanceado y guardar CSV
rows = [] #Convierte en columna las reseñas
for lbl, items in reservoir.items():
    rows.extend(items)
random.shuffle(rows)
#Crear dataframe y guardar csv
df_bal = pd.DataFrame(
    rows,
    columns=["text", "label", "overall", "mention_price", "mention_quality", "mention_shipping"])
print("\nConteo por clase en la muestra balanceada:")
print(df_bal["label"].value_counts())

df_bal.to_csv(out_csv, index=False)
print(f"\nArchivo guardado: {out_csv}  (filas: {len(df_bal):,})")
df_bal.head(3)

Leyendo el archivo JSON Lines…

 Muestras completas después de leer 101,415.
Lectura finalizada.

Conteo por clase en la muestra balanceada:
label
positive    10000
neutral     10000
negative    10000
Name: count, dtype: int64

Archivo guardado: amazon_reviews_limpio_balanceado.csv  (filas: 30,000)


,text,label,overall,mention_price,mention_quality,mention_shipping
0,"Bought this for my husband (iphone 4), he had ...",positive,5.0,False,True,False
1,For under $20 I wasn't expecting much but was ...,positive,5.0,True,True,False
2,Thanks,neutral,3.0,False,False,False


Descarga CSV

In [ ]:
from google.colab import files
files.download("amazon_reviews_limpio_balanceado.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Conclusiones de Limpieza y Balanceo de Datos**

1. **Estructura inicial del dataset**  
   El dataset original provenía de un archivo JSON, que fue transformado a formato CSV para facilitar su manejo y análisis en herramientas como Google Colab, asi como, para el uso adecuado de la BD dentro de colab se optó por reducir el numero de registros mediante una adecuada limpieza y balanceo del mismo.

2. **Limpieza aplicada**  
   - Se eliminaron registros con valores nulos y duplicados, asegurando que la información sea consistente.  
   - Se normalizaron los nombres de columnas para mantener una estructura clara.  
   - Se removieron caracteres especiales y espacios innecesarios en las reseñas para evitar problemas en el procesamiento de texto.

3. **Balanceo de clases**  
   - Inicialmente, las clases de sentimiento (positivo, neutral y negativo) presentaban un desbalance.  
   - Se aplicó un muestreo equilibrado para que cada clase cuente con la misma cantidad de registros.  
   - El dataset final contiene **10,000 registros por clase**, evitando sesgos durante el entrenamiento del modelo.

4. **Dataset final**  
   - El dataset resultante está **limpio, balanceado y listo** para su uso en análisis exploratorio y modelado.  
   - Se guardó en formato **CSV** en la carpeta `data/raw` para mantener un flujo de trabajo organizado.
